# Population by Ancestry, Hispanic and Asian Origin

The purpose of this notebook is to use the 2023 5-Year American Community Survey's data files on populations by self-reported ancestry, hispanic origin and asian origin to figure out what proportions of those groups reside in the city's City Council districts. 
<br>
<br>
<b> The Census's API can be used to retrieve the data, but I downloaded the files instead. If taking this route, choose all census tracts within NYS. These are the files: </b>
- B04006 | People Reporting Ancestry
- B03001 | Hispanic Or Latino Origin By Specific Origin
- B02018 | Asian Alone By Selected Groups

<b> After downloading, follow these steps to clean the files: </b>
- Rename the zipped folders and data files. 
- Do a TEXTSPLIT() of the "Geography" column on "US" and of the "Geographic Area Name" column on ";". 
- Clean up the columns so that you have just the census tracts and county names remaining from the step above.
- Filter for only Bronx, Richmond, Queens, Kings and New York County. Paste the selection onto another sheet. You should be left with 2,327 rows, which equals the number of census tracts in NYC. 
- Rename the columns, remove the margin of error columns, and correct the data types of each column.
- Optional: Get rid of columns in the cleaned sheet that wouldn't be useful in the actual analysis (e.g. 'Other Groups' or 'Not Specified'), but keep them in the raw sheets.



In [1]:
## import libraries
import pandas as pd
import geopandas as gpd

In [2]:
## set viz options
pd.set_option('display.max_rows', 500)

In [3]:
## import asian, hispanic and ancestry data from ACS 5Y for all five NYC counties. 
asian = pd.read_excel("../input/demographics/ethnicity/B02018_2023.xlsx")
hispanic = pd.read_excel('../input/demographics/ethnicity/b03001_2023.xlsx', sheet_name = 'cleaned_filtered')
ancestry = pd.read_excel('../input/demographics/ethnicity/b04006_2023.xlsx', sheet_name = 'cleaned_filtered')

In [4]:
## import crosswalk, which matches 2020 census tracts with 2023 council districts
crosswalk = pd.read_csv('../input/crosswalks/ct20-to-cd23-crosswalk.csv',
                        dtype = {'ct':'str', 'cd':'str'})

In [5]:
## import other geom files
old_demographics_file = gpd.read_file('../input/GIS/old_gis/demographics.json')
old_centroid_file = gpd.read_file('../input/GIS/old_gis/demographics-centroids.json')

#### reshape the asian, hispanic and ancestry data

In [6]:
## reshape the datasets so that we have only three columns
asian.columns = asian.columns.str.strip()

In [7]:
## define id variables and value variables
asian_id_vars = ['ct']
asian_value_vars = [col for col in asian.columns if col not in ['ct', 'county']]

In [8]:
## reshape
melted_asian = asian.melt(id_vars=asian_id_vars, 
                    value_vars=asian_value_vars, 
                    var_name='ethnicity', 
                    value_name='population')

In [9]:
## reshape the datasets so that we have only three columns
hispanic.columns = hispanic.columns.str.strip()

In [10]:
## identify variables
## define id variables and value variables
hisp_id_vars = ['ct']
hisp_value_vars = [col for col in hispanic.columns if col not in ['ct', 'county']]

In [11]:
## reshape hispanic dataset
melted_hispanic = hispanic.melt(id_vars=hisp_id_vars, 
                    value_vars=hisp_value_vars, 
                    var_name='ethnicity', 
                    value_name='population')

In [12]:
## reshape the datasets so that we have only three columns
ancestry.columns = ancestry.columns.str.strip()

In [13]:
## identify variables
## define id variables and value variables
ancestry_id_vars = ['ct']
ancestry_value_vars = [col for col in ancestry.columns if col not in ['ct', 'county']]

In [14]:
## reshape ancestry dataset
melted_ancestry = ancestry.melt(id_vars=ancestry_id_vars, 
                    value_vars=ancestry_value_vars, 
                    var_name='ethnicity', 
                    value_name='population')

In [15]:
## combine them all using concat
combined = pd.concat([melted_ancestry, melted_asian, melted_hispanic], ignore_index = True)

In [50]:
## change ct dtype
combined['ct'] = combined['ct'].astype('object')

#### get the top ethnicity in each census tract

In [16]:
## get all unique tracts
all_tracts = combined[['ct']].drop_duplicates()

In [17]:
## get the top ethnicity per tract, only if population > 0
top_per_tract = (
    combined[combined['population'] != 0]
    .groupby(['ct','ethnicity'])['population']
    .sum()
    .reset_index()
    .sort_values(by=['ct', 'population'], ascending=[True, False])
    .groupby('ct')
    .head(1)  # top ethnicity per tract
    .reset_index(drop=True)
)

In [18]:
## merge with all tracts to ensure every tract appears
add_nulls = all_tracts.merge(top_per_tract, on='ct', how='left')

In [19]:
## change 'ct' column dtype 
add_nulls['ct'] = add_nulls['ct'].astype('str')

In [20]:
## checking rows...
add_nulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327 entries, 0 to 2326
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ct          2327 non-null   object 
 1   ethnicity   2242 non-null   object 
 2   population  2242 non-null   float64
dtypes: float64(1), object(2)
memory usage: 54.7+ KB


In [21]:
add_nulls.head()

,ct,ethnicity,population
0,36005000100,Puerto Rican,197.0
1,36005000200,Puerto Rican,1273.0
2,36005000400,Puerto Rican,2077.0
3,36005001600,Puerto Rican,1843.0
4,36005001901,Dominican,370.0


In [22]:
## rename columns 
renamed_columns = add_nulls.rename(columns = {'ethnicity':'lar_grp',
                                              'population':'lar_pop'})

#### add geometries

In [23]:
## first do regular geometries 
demographics = renamed_columns.merge(old_demographics_file[['ct','cd','nta','geometry']],
                                    on = 'ct',
                                    how = 'left')

In [24]:
## make into a geo dataframe
demo_gdf = gpd.GeoDataFrame(demographics,
                            geometry = demographics.geometry,
                            crs = 4326)

In [25]:
## write to file
demo_gdf.to_file('../output/demographics/demographics.geojson', driver = 'GeoJSON')

In [26]:
## now centroids
demo_centroids = renamed_columns.merge(old_centroid_file[['ct','cd','nta','geometry']],
                                    on = 'ct',
                                    how = 'left')

In [27]:
## make into geodataframe
centroid_gdf = gpd.GeoDataFrame(demo_centroids,
                                geometry = demo_centroids.geometry,
                                crs = 4326)

In [28]:
## write to file
centroid_gdf.to_file('../output/demographics/demographic-centoids.geojson', driver = 'GeoJSON')

#### find top three-largest geometries by council district

In [54]:
## remove any whitespace from the ct columns
## checking for and removing whitespace
combined['ct'] = combined['ct'].astype(str).str.strip().str.replace('\xa0', '', regex=True)
crosswalk['ct'] = crosswalk['ct'].astype(str).str.strip().str.replace('\xa0', '', regex=True)

In [101]:
## merge the ethnicties data with the crosswalk so we get the CDs
top_three = combined.merge(crosswalk[['ct','cd']],
                           on = 'ct',
                           how = 'left')

In [106]:
## group ethnic populations by council district
grouped = top_three.groupby(['cd','ethnicity'])['population'].sum().reset_index()

In [107]:
## take a peak
grouped

,cd,ethnicity,population
0,1,Afghan,15
1,1,African,497
2,1,Albanian,217
3,1,Alsatian,0
4,1,Arab Arab,83
...,...,...,...
7594,9,W Indians Trinidadian and Tobagonian,1198
7595,9,W Indians West Indian,1447
7596,9,Welsh,348
7597,9,Yugoslavian,0
